In [ ]:
import os
os.chdir('C:\\Github')

## 說明：
- 前一步我們以 List 形式儲存物件辨識的結果：
```
[
int(position[0:2]), int(position[2:4]), \ # 該 frame 對應的 x, y 座標
frame_id, \ # 該 frame 的編號
boxes.numpy()[0],\ # bounding box 的點座標 (左上&右下)
scores.numpy()[0],\ # 信心分數
classes.numpy()[0],\ # 物件種類
valid_detections.numpy()[0],\ # 物件數量 
vid.get(0),\ # 對應的影片秒數
0 # 面向方向，詳見"實驗地圖與方向定義(座標).png"
]    
```
- 在這一步會進一步轉換它們為 image fingerprint 形式，結果就像
# ![](https://i.imgur.com/IqcWGxd.png)

- 同時，我們論文中探討物件遮擋問題對結果的影響，在這裡可以用 missing_rate_list 來實作。

In [ ]:
#from absl import app, flags, logging
#from absl.flags import FLAGS

import os
from os.path import join
import pickle
import numpy as np
import csv
import pickle

from random import seed
from random import random

# YOLO pre-trained model 的物件列表
obj_list = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane',\
'bus', 'train', 'truck', 'boat', 'traffic light',\
'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',\
'cat', 'dog', 'horse', 'sheep', 'cow',\
'elephant', 'bear', 'zebra', 'giraffe', 'backpack',\
'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',\
'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',\
'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',\
'wine glass', 'cup', 'fork', 'knife', 'spoon',\
'bowl', 'banana', 'apple', 'sandwich', 'orange',\
'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',\
'cake', 'chair', 'sofa', 'potted plant', 'bed',\
'dining table', 'toilet', 'tvmonitor', 'laptop',\
'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',\
'oven', 'toaster', 'sink', 'refrigerator', 'book',\
'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',\
'toothbrush']

class obj_info:
    def __init__(self, x, y, frame_id, Box, Score, Class):
        self.x     = x 
        self.y     = y 
        self.frame_id = frame_id
        self.Box   = Box  # [左上y, 左上x, 右下y, 右下x]
        self.Score = Score 
        self.Class = Class 
    def Show(self):
        print(f"Position: {self.x}, {self.y}, Frame:{self.frame_id}\nBox: {self.Box}\nScore: {self.Score}\nClass: {obj_list[int(self.Class)]} ({self.Class})")

class frame_info:
    def __init__(self, x, y, frame_id, heading, Time, obj_list):
        self.x     = x 
        self.y     = y
        self.heading  = heading
        self.Time   = Time
        self.frame_id = frame_id
        self.obj_list = obj_list
    def Show(self):
        print(f"There r {len(self.obj_list)} objects in this frame ({self.x},{self.y}) \ndir: {self.heading}\nTime: {self.Time}\nID: {self.frame_id}")

def unique_obj(data):
    obj_set = set([])
    
    for video in data: # for every frame
        for frame in video: # for every bounding box
            for box in frame.obj_list:
                obj_set.add(box.Class)
            
    with open(set_raw_path, 'wb') as f:
        pickle.dump(list(sorted(obj_set)), f)
        
    return list(sorted(obj_set))

# coordinate to number
table = [\
        [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41]]

## 參數
1. phones、test_dir、walks: 用來組成讀取辨識結果的路徑
```
f'{test_dir}/{walk}/{phone}/detect_res_1.pickle'
```
2. missing_rate_list: missing_rate 表示物件被忽略的比例，也就是論文中的 R，將你感興趣的 R 放在 missing_rate_list 中，就可以生成相對應的 image fingerprint，例如
    ```
    missing_rate_list = [0.1, 0.3, 0.5]
    ```
可以生成 "image_fingerprint_1.csv", "image_fingerprint_3.csv", "image_fingerprint_5.csv"



In [ ]:
# Input Parameters
phones = ['hTCU11','sharp025','hTCU19e','GalaxyA51'] # 
test_dir = './Data/0318_92589_test/'
walks = [
    'one_way_walk_1','one_way_walk_2','one_way_walk_3','one_way_walk_4','one_way_walk_5','one_way_walk_6','one_way_walk_7','one_way_walk_8',\
    'round_trip_walk_2','round_trip_walk_3','round_trip_walk_4',\
    'stationary_1',\
    'freewalk_1','freewalk_2','freewalk_3','freewalk_4','freewalk_5','freewalk_6','freewalk_7','freewalk_8','freewalk_9'
]


missing_rate_list = [1]#
#missing_rate_list = [0.6, 0.8]

for obj_missing_rate in missing_rate_list:

    for walk in walks:
        for phone in phones:
            res_path = f'{test_dir}/{walk}/{phone}/detect_res_1.pickle'
            set_raw_path = f'./Data/0318_92589_train_image/image_obj_set.pickle'

            # Output
            info_raw_path = f'{test_dir}/{walk}/{phone}/92589_image_testset.pickle'
            fingerprint_raw_path = f'{test_dir}/{walk}/{phone}/image_info.csv'
            #info_raw_path = f'{test_dir}/{walk}/{phone}/92589_image_testset_{int(10*obj_missing_rate)}.pickle'
            #fingerprint_raw_path = f'{test_dir}/{walk}/{phone}/image_info_{int(10*obj_missing_rate)}.csv'

            print(f'read result from: {res_path}\nread set from: {set_raw_path}')
            print(f'output info to: {info_raw_path}\noutput fingerprint to: {fingerprint_raw_path}')

            with open(res_path,'rb') as f:
                test_res = pickle.load(f) 

            testset_info = []
            for video in test_res:
                video_info = []
                for frame in video: # every box info
                    curr_x = frame[0]
                    curr_y = frame[1]
                    curr_frame_id = frame[2]
                    curr_sec = frame[7]
                    curr_heading = frame[8]
                    curr_objs = []
                    for index in range(frame[6]): # all obj in frame
                        Box = frame[3][index]
                        Score = frame[4][index]
                        Class = frame[5][index]
                        box_info = obj_info(curr_x, curr_y, curr_frame_id, Box, Score, Class)
                        if random() <= obj_missing_rate:
                            curr_objs.append(box_info)
                    curr_frame_info = frame_info(curr_x, curr_y, curr_frame_id, curr_heading, curr_sec, curr_objs)
                    video_info.append(curr_frame_info)
                testset_info.append(video_info)

            # save file raw
            with open(info_raw_path,'wb') as f:
                pickle.dump(testset_info,f)

            with open(set_raw_path,'rb') as f:
                obj_set = pickle.load(f)

            # generate fingerprint (csv)
            with open(fingerprint_raw_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                header = []

                header.append('label')
                header.append('frame_id')
                header.append('heading')
                header.append('second')
                header.append('object_count')
                for i in range(25) :
                    header.append(f'object_{i}_class')
                    header.append(f'object_{i}_score')
                    header.append(f'object_{i}_x')
                    header.append(f'object_{i}_y')
                    header.append(f'object_{i}_w')
                    header.append(f'object_{i}_h')

                writer.writerow(header)

                for video in testset_info: # for every video
                    for frame in video: # for every frame

                        frame_x = frame.x
                        frame_y = frame.y
                        fingerprint = [0.0]*155
                        object_info_list = []
                        object_count = 0.0

                        fingerprint[0] = table[frame_y][frame_x]
                        fingerprint[1] = frame.frame_id # last one
                        fingerprint[2] = frame.heading
                        fingerprint[3] = frame.Time

                        for obj in frame.obj_list:
                            obj_class = int(obj.Class)
                            if obj_class in obj_set :
                                object_count += 1

                                width = np.abs(obj.Box[1] - obj.Box[3])
                                height = np.abs(obj.Box[0] - obj.Box[2])
                                cx = (obj.Box[1] + obj.Box[3])/2
                                cy = (obj.Box[0] + obj.Box[2])/2
                                score = obj.Score

                                fingerprint[int(object_count-1)*6+5] = obj_class
                                fingerprint[int(object_count-1)*6+6] = score
                                fingerprint[int(object_count-1)*6+7] = cx
                                fingerprint[int(object_count-1)*6+8] = cy
                                fingerprint[int(object_count-1)*6+9] = width
                                fingerprint[int(object_count-1)*6+10] = height

                        fingerprint[4] = object_count
                        writer.writerow(fingerprint)
            print(f'testing fingerprint done: {fingerprint_raw_path}')